In [22]:
import numpy as np
from tqdm import tqdm

import pandas as pd

from fuzzywuzzy import fuzz
import textdistance

from itertools import product,combinations_with_replacement,combinations,permutations

In [4]:
datapath = storepath = '..\\..\\..\\..\\TF_data\\ADMIN\\v2\\04_DATA\\STF.xlsx'

In [5]:
bf = pd.read_excel(datapath,index_col=0,parse_dates=['Date_booked','Date_ordered'])

In [7]:
bf.columns

Index(['OrderAccount', 'Date_booked', 'Date_ordered', 'Text_transaction',
       'Use', 'Contact', 'AccNum', 'BIC', 'Value_transaction', 'Currency',
       'Information', 'SpendingType', 'ExpenseGroup'],
      dtype='object')

In [60]:
# First, remove irrelevant columns

df = bf[[
    'OrderAccount',
    'Date_booked',
    'Date_ordered',
    'Text_transaction',
    'Use',
    'Contact',
    'AccNum',
    'BIC',
    'Value_transaction',
    #'Currency',
    #'Information',
    #'SpendingType',
    #'ExpenseGroup',
]]

In [70]:
df.head(10)

,OrderAccount,Date_booked,Date_ordered,Text_transaction,Use,Contact,AccNum,BIC,Value_transaction
0,DE98773501100020557849,2019-10-02,2019-10-01,BARGELDAUSZAHLUNG,2019 10 01T22 08 Debitk 5 2021 12,DECHSENDOR SPARKASSE ERLANGEN DE,DE63763500009000481424,BYLADEM1ERH,-50.00
1,DE98773501100020557849,2019-09-30,2019-10-01,ABSCHLUSS,Abrechnung 30 09 2019 siehe Anlage,NaN,0020557849,77350110,0.00
2,DE98773501100020557849,2019-09-30,2019-10-01,ENTGELTABSCHLUSS,Entgeltabrechnung siehe Anlage,NaN,0000000000,77350110,-4.50
3,DE98773501100020557849,2019-09-30,2019-09-30,DAUERAUFTRAG,Miete fuer Haesschen du bist toll,Elaine Fernandez,DE46763500000044116613,BYLADEM1ERH,-235.00
4,DE98773501100020557849,2019-09-30,2019-09-30,FOLGELASTSCHRIFT,V Nr 47589 09 19 Beitrag 24 90,SLF Sportland Franken GmbH Co KG,DE88763500000000062725,BYLADEM1ERH,-24.90
5,DE98773501100020557849,2019-09-30,2019-09-30,FOLGELASTSCHRIFT,300919 15194815 610971995391 SIEMENS CASINO CA...,Siemens Aktiengesellschaft,DE53700700100203008800,DEUTDEMMXXX,-66.89
6,DE98773501100020557849,2019-09-30,2019-09-30,FOLGELASTSCHRIFT,Mobilfunk Kundenkonto 0051900308 RG 2922363700...,Telekom Deutschland GmbH Landgrabenweg 151,DE68700202700667302269,HYVEDEMMXXX,-64.90
7,DE98773501100020557849,2019-09-30,2019-09-30,FOLGELASTSCHRIFT,VST 458207 62 00 EUR A19 656666,ESTW ERLANGERSTADTWERKEAG,DE06763200720004536703,HYVEDEMM417,-62.00
8,DE98773501100020557849,2019-09-30,2019-09-30,FOLGELASTSCHRIFT,BEITRAG BIS 09 19 MANDATSREF 4005172401 GLAEUB...,IGM Erlangen,DE28500500000083044008,HELADEFFXXX,-2.05
9,DE98773501100020557849,2019-09-27,2019-09-27,LOHN GEHALT,lohn gehalt,Siemens AG,DE90700500000002055382,BYLADEMMXXX,2539.35


In [62]:
idf0 = pd.DataFrame(list(combinations(df.index,2))).rename(columns={0:'Index0',1:'Index1'})

In [71]:
cf = idf0.merge(right=df[['Date_ordered','Date_booked','Value_transaction']].reset_index().rename(columns={
    'index':'Index0',
    'Date_ordered':'Date_ordered0',
    'Date_booked':'Date_booked0',
    'Value_transaction':'Value_transaction0'
}),how='left',on='Index0').merge(
    right=df[['Date_ordered','Date_booked','Value_transaction']].reset_index().rename(columns={
    'index':'Index1',
    'Date_ordered':'Date_ordered1',
    'Date_booked':'Date_booked1',
    'Value_transaction':'Value_transaction1'
}),how='left',on='Index1'
)

In [72]:
cf

,Index0,Index1,Date_ordered0,Date_booked0,Value_transaction0,Date_ordered1,Date_booked1,Value_transaction1
0,0,1,2019-10-01,2019-10-02,-50.00,2019-10-01,2019-09-30,0.00
1,0,2,2019-10-01,2019-10-02,-50.00,2019-10-01,2019-09-30,-4.50
2,0,3,2019-10-01,2019-10-02,-50.00,2019-09-30,2019-09-30,-235.00
3,0,4,2019-10-01,2019-10-02,-50.00,2019-09-30,2019-09-30,-24.90
4,0,5,2019-10-01,2019-10-02,-50.00,2019-09-30,2019-09-30,-66.89
...,...,...,...,...,...,...,...,...
417236,910,912,2019-11-22,2019-11-22,-1480.02,2019-11-18,2019-11-20,-193.86
417237,910,913,2019-11-22,2019-11-22,-1480.02,2019-11-20,2019-11-20,-367.15
417238,911,912,2019-11-21,2019-11-21,-29.89,2019-11-18,2019-11-20,-193.86
417239,911,913,2019-11-21,2019-11-21,-29.89,2019-11-20,2019-11-20,-367.15


In [73]:
cf['Booked - Time Delta'] = (cf['Date_booked0']-cf['Date_booked1']).dt.days

In [74]:
cf['Ordered - Time Delta'] = (cf['Date_ordered0']-cf['Date_ordered1']).dt.days

In [75]:
cf['Value Delta'] = (cf['Value_transaction0']-cf['Value_transaction1']).abs()

In [79]:
cf[(cf['Booked - Time Delta']==0)&(cf['Ordered - Time Delta']==0)&(cf['Value Delta']==0)]

,Index0,Index1,Date_ordered0,Date_booked0,Value_transaction0,Date_ordered1,Date_booked1,Value_transaction1,Booked - Time Delta,Ordered - Time Delta,Value Delta
548,0,549,2019-10-01,2019-10-02,-50.00,2019-10-01,2019-10-02,-50.00,0,0,0.0
1461,1,550,2019-10-01,2019-09-30,0.00,2019-10-01,2019-09-30,0.00,0,0,0.0
2373,2,551,2019-10-01,2019-09-30,-4.50,2019-10-01,2019-09-30,-4.50,0,0,0.0
4193,4,552,2019-09-30,2019-09-30,-24.90,2019-09-30,2019-09-30,-24.90,0,0,0.0
5102,5,553,2019-09-30,2019-09-30,-66.89,2019-09-30,2019-09-30,-66.89,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
408330,780,781,2020-03-03,2020-03-03,-3.99,2020-03-03,2020-03-03,-3.99,0,0,0.0
410913,801,802,2020-02-18,2020-02-18,-3.99,2020-02-18,2020-02-18,-3.99,0,0,0.0
416941,889,890,2019-12-03,2019-12-03,-0.99,2019-12-03,2019-12-03,-0.99,0,0,0.0
416943,889,892,2019-12-03,2019-12-03,-0.99,2019-12-03,2019-12-03,-0.99,0,0,0.0
